In [30]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
from torch_geometric.nn import GATConv, SAGPooling,global_mean_pool , global_max_pool 
from sklearn.metrics import mean_absolute_error
from torch import optim

## fingat源码的关于编码的代码

In [31]:
class AttentionBlock(nn.Module):
    def __init__(self,time_step,dim):
        super(AttentionBlock, self).__init__()
        self.attention_matrix = nn.Linear(time_step, time_step)

    def forward(self, inputs):
        inputs_t = torch.transpose(inputs,2,1) # (batch_size, input_dim, time_step)
        attention_weight = self.attention_matrix(inputs_t)
        attention_probs = F.softmax(attention_weight,dim=-1)
        attention_probs = torch.transpose(attention_probs,2,1)
        attention_vec = torch.mul(attention_probs, inputs)
        attention_vec = torch.sum(attention_vec,dim=1)
        return attention_vec, attention_probs

class SequenceEncoder(nn.Module):
    def __init__(self,input_dim,time_step,hidden_dim):
        super(SequenceEncoder, self).__init__()
        self.encoder = nn.GRU(input_size=input_dim,hidden_size=hidden_dim,num_layers=1,batch_first=True)
        self.attention_block = AttentionBlock(time_step,hidden_dim) 
        self.dropout = nn.Dropout(0.2)
        self.dim = hidden_dim
    
    def forward(self,seq):
        '''
        inp : torch.tensor (batch,time_step,input_dim)
        '''
        seq_vector,_ = self.encoder(seq)
        seq_vector = self.dropout(seq_vector)
        attention_vec, attention_probs = self.attention_block(seq_vector)
        attention_vec = attention_vec.view(-1,1,self.dim) # prepare for concat
        return attention_vec, attention_probs

## 导入数据

In [32]:
with open('./datasets3/sp500_data.pkl', "rb") as f:
    data = pickle.load(f)

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
train_x = data['train']['x1'][:,:,:,1:]
train_x.shape

(1577, 475, 7, 29)

In [35]:
test_x = data['test']['x1'][:,:,:,1:]
test_x.shape

(387, 475, 7, 29)

In [36]:
train_y = data['train']['y_return ratio']
train_y.shape

(1577, 475)

In [37]:
test_y = data['test']['y_return ratio']
test_y.shape

(387, 475)

## 设立pytorch的dataloader

In [38]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

class StockDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def create_dataloader(x, y, batch_size):
    dataset = StockDataset(x, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [39]:
dataloader = create_dataloader(train_x, train_y, batch_size=16)

In [40]:
for batch_idx, (seq, target) in enumerate(dataloader):
    print(batch_idx)
    print(seq.shape)
    print(target.shape)
    print("=========================")

0
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
1
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
2
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
3
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
4
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
5
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
6
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
7
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
8
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
9
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
10
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
11
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
12
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
13
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
14
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
15
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
16
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
17
torch.Size([16, 475, 7, 29])
torch.Size([16, 475])
18
torch.Size([16, 475, 7, 29])
torch.

## 建立模型，设置优化器、损失，进行训练

In [41]:
# 建立
sequence_encoder = SequenceEncoder(input_dim=29, time_step=7, hidden_dim=64)

# 定义损失和优化
criterion = nn.MSELoss()  # 因为是回归问题，所以我们使用均方误差损失
optimizer = Adam(sequence_encoder.parameters(), lr=0.001)

In [42]:
# 5. 训练模型
def train_model(model, dataloader, epochs):
    min_loss = float('inf')  # 初始最小损失设为正无穷大
    model = model.to(device)
    model.train()
    for epoch in range(epochs):
        for batch_idx, (seq, target) in enumerate(dataloader):
            seq = seq.to(device).float()
            target = target.to(device).float()
            optimizer.zero_grad()
            seq = seq.view(-1, 7, 29)  # Reshape to (batch_size * num_stocks, time_step, input_dim)
#             print(seq.shape)  #  [5984, 7, 30]
            target = target.view(-1,1)  # Flatten target to have shape (batch_size * num_stocks)
#             print(target.shape)  #  [5984, 1]
            attention_vec, attention_probs = model(seq)
            attention_vec = attention_vec.squeeze()  # Remove extra dimensions to match target shape
            loss = criterion(attention_vec, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 20 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss {loss.item()}')
        if loss.item() < min_loss:  # 当前损失小于记录的最小损失就保存
            min_loss = loss.item() 
            torch.save(model, './output/sequence_encoder-1202.pkl') 
            print('save!')

# 开始训练
train_model(sequence_encoder, dataloader, epochs=50)

D:\_Application\anaconda3\envs\Pytorch\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([7600, 1])) that is different to the input size (torch.Size([7600, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Batch 0, Loss 0.01323576271533966
Epoch 0, Batch 20, Loss 0.005960122682154179
Epoch 0, Batch 40, Loss 0.0030632023699581623
Epoch 0, Batch 60, Loss 0.0022106030955910683
Epoch 0, Batch 80, Loss 0.0010974928736686707


D:\_Application\anaconda3\envs\Pytorch\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([4275, 1])) that is different to the input size (torch.Size([4275, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


save!
Epoch 1, Batch 0, Loss 0.0010343975154682994
Epoch 1, Batch 20, Loss 0.0005980537971481681
Epoch 1, Batch 40, Loss 0.0004282526788301766
Epoch 1, Batch 60, Loss 0.0004119219083804637
Epoch 1, Batch 80, Loss 0.00034123024670407176
save!
Epoch 2, Batch 0, Loss 0.0002599103609099984
Epoch 2, Batch 20, Loss 0.0003521883918438107
Epoch 2, Batch 40, Loss 0.00031057122396305203
Epoch 2, Batch 60, Loss 0.0002388919674558565
Epoch 2, Batch 80, Loss 0.0003080145106650889
save!
Epoch 3, Batch 0, Loss 0.0003723150584846735
Epoch 3, Batch 20, Loss 0.0002213188272435218
Epoch 3, Batch 40, Loss 0.0002564689493738115
Epoch 3, Batch 60, Loss 0.0011172927916049957
Epoch 3, Batch 80, Loss 0.00043539382750168443
Epoch 4, Batch 0, Loss 0.0005616576527245343
Epoch 4, Batch 20, Loss 0.0003458669234532863
Epoch 4, Batch 40, Loss 0.00039282639045268297
Epoch 4, Batch 60, Loss 0.0002571314398664981
Epoch 4, Batch 80, Loss 0.0003368014586158097
save!
Epoch 5, Batch 0, Loss 0.00024942727759480476
Epoch 5, B

Epoch 36, Batch 0, Loss 0.0015290678711608052
Epoch 36, Batch 20, Loss 0.00040607177652418613
Epoch 36, Batch 40, Loss 0.0003908427606802434
Epoch 36, Batch 60, Loss 0.00029335578437894583
Epoch 36, Batch 80, Loss 0.00040692303446121514
Epoch 37, Batch 0, Loss 0.00027693953597918153
Epoch 37, Batch 20, Loss 0.0014877233188599348
Epoch 37, Batch 40, Loss 0.0002754738961812109
Epoch 37, Batch 60, Loss 0.00045792327728122473
Epoch 37, Batch 80, Loss 0.0003056888817809522
Epoch 38, Batch 0, Loss 0.0004344023473095149
Epoch 38, Batch 20, Loss 0.0002203289041062817
Epoch 38, Batch 40, Loss 0.0002881785330828279
Epoch 38, Batch 60, Loss 0.0007842259365133941
Epoch 38, Batch 80, Loss 0.0005404292605817318
Epoch 39, Batch 0, Loss 0.00043660664232447743
Epoch 39, Batch 20, Loss 0.00032409184495918453
Epoch 39, Batch 40, Loss 0.0003904725017491728
Epoch 39, Batch 60, Loss 0.00039914780063554645
Epoch 39, Batch 80, Loss 0.0002900398976635188
Epoch 40, Batch 0, Loss 0.0002769621496554464
Epoch 40, 

## 测试数据，加载node和edge

In [43]:
test_x.shape

(387, 475, 7, 29)

In [44]:
test_y.shape

(387, 475)

In [45]:
test = test_x[0,:,:,:]
test.shape

(475, 7, 29)

In [46]:
# torch.Size([16, 480, 7, 29])
# torch.Size([16, 480])
new_week_data = test_x[0,:,:,:]
# 评估模型
sequence_encoder.eval()

# 不需要计算梯度
with torch.no_grad():
    input = torch.Tensor(new_week_data).view(-1,7,29).to(device)
    print(input.size())
    attention_vecs, attention_probs = sequence_encoder(input)
    print(attention_vecs.size())
    print(attention_probs.size())

torch.Size([475, 7, 29])
torch.Size([475, 1, 64])
torch.Size([475, 7, 64])


In [47]:
attention_vecs.size()

torch.Size([475, 1, 64])

In [48]:
inner_edge = np.array(np.load("./datasets3/inner_edge.npy"))
inner_edge

array([[  0,   0],
       [  0,  23],
       [  0,  25],
       ...,
       [426, 426],
       [426, 447],
       [447, 447]])

In [49]:
inner_edge = torch.tensor(inner_edge.T, dtype=torch.int64).to(device)
inner_edge

tensor([[  0,   0,   0,  ..., 426, 426, 447],
        [  0,  23,  25,  ..., 426, 447, 447]], device='cuda:0')

In [50]:
inner_edge.size()

torch.Size([2, 9945])

## 一周内、不同股票间的gat代码

In [51]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATModel(torch.nn.Module):
    def __init__(self, hidden_dim):
        super(GATModel, self).__init__()
        self.conv1 = GATConv(hidden_dim,hidden_dim)

    def forward(self, x, edge_index):
        # 输入层
        x, attention_weights = self.conv1(x, edge_index, return_attention_weights=True)
        x = F.elu(x)

        return x, attention_weights


In [52]:
# node和edge
nodes = attention_vecs.squeeze(1)  # 假设这是你的节点特征
print(nodes.size())
edges = inner_edge  # 假设这是你的边索引
print(edges.size())

# 创建模型
hidden_num = 64
model = GATModel(hidden_num).to(device)

# 前向传播
model.train()
out, gat_attention = model(nodes, edges)


torch.Size([475, 64])
torch.Size([2, 9945])


In [53]:
out.size()

torch.Size([475, 64])

In [54]:
gat_attention

(tensor([[  0,   0,   0,  ..., 472, 473, 474],
         [ 23,  25,  26,  ..., 472, 473, 474]], device='cuda:0'),
 tensor([[0.5009],
         [0.3342],
         [0.2507],
         ...,
         [0.0333],
         [0.0227],
         [0.0833]], device='cuda:0', grad_fn=<DivBackward0>))

In [56]:
gat_attention[1].size()

torch.Size([9945, 1])

## 遍历每周gat图得到g_i和a_i

In [35]:
inner_edge = np.array(np.load("/openbayes/input/input0/inner_edge.npy"))
inner_edge = torch.tensor(inner_edge.T, dtype=torch.int64).to(device)
inner_edge

tensor([[  0,   0,   0,  ..., 426, 426, 447],
        [  0,  23,  25,  ..., 426, 447, 447]], device='cuda:0')

In [46]:
best_model = torch.load('./output/sequence_encoder.pkl')

In [38]:
test_x.shape

(390, 480, 7, 29)

In [53]:
gat_result = {}
encoder_result = {}
for i in range(test_x.shape[0]):  # test_x.shape[0]
    week_data = torch.Tensor(test_x[i,:,:,:]).view(-1,7,29).to(device)
    attention_vecs, attention_probs = best_model(week_data)
    encoder_result[f'test_week{i+1}'] = attention_vecs.squeeze(1).detach().cpu().numpy()
    nodes = attention_vecs.squeeze(1)
    # print(nodes.size())
    edges = inner_edge  # 假设这是你的边索引
    # print(edges.size())
    gat = GATModel(num_node_features=64, num_hidden_units=8, num_classes=8).to(device)
    # 前向传播
    gat.train()
    out = gat(nodes, edges)
    gat_result[f'test_week{i+1}'] = out.detach().cpu().numpy()
    # print(out.size())

In [54]:
gat_result['test_week1'].shape

(480, 8)

In [55]:
encoder_result['test_week1'].shape

(480, 64)

In [57]:
import pickle

with open('./output/g_i.pkl', 'wb') as file:
    pickle.dump(gat_result, file)
file.close()

with open('./output/a_i.pkl', 'wb') as file:
    pickle.dump(encoder_result, file)
file.close()